In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,30300.00,-0.23,0.31,0.48,0.06,0.07,0.29,0.56,-0.03,0.10,...,0.01,0.01,-0.01,0.02,0.01,-0.00,0.0,-0.00,0.00,-0.00
3,-0.23,20591.55,0.01,-0.34,0.29,-0.46,0.07,0.13,-0.07,-0.04,...,-0.01,0.02,0.00,0.01,0.00,-0.00,0.0,-0.00,-0.00,-0.00
4,0.31,0.01,15123.64,0.53,0.17,-0.02,0.18,-0.18,0.21,0.00,...,-0.03,0.00,0.01,-0.01,0.00,0.00,0.0,-0.00,0.00,0.00
5,0.48,-0.34,0.53,10582.97,-0.37,0.33,-0.03,-0.09,0.03,0.02,...,-0.01,-0.01,0.00,-0.00,0.00,0.00,0.0,-0.00,0.00,0.00
6,0.06,0.29,0.17,-0.37,6911.37,-0.01,-0.02,-0.01,-0.09,0.06,...,0.01,-0.00,0.01,0.00,0.00,0.00,-0.0,0.00,-0.00,-0.00
7,0.07,-0.46,-0.02,0.33,-0.01,4179.84,-0.01,0.01,0.05,-0.02,...,-0.02,0.01,-0.01,0.01,-0.00,0.00,0.0,-0.00,0.00,-0.00
8,0.29,0.07,0.18,-0.03,-0.02,-0.01,2594.32,0.02,-0.08,0.06,...,0.00,0.01,-0.00,0.00,-0.00,0.00,-0.0,-0.00,-0.00,-0.00
9,0.56,0.13,-0.18,-0.09,-0.01,0.01,0.02,1466.16,-0.04,-0.02,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.0,0.00,-0.00,-0.00
10,-0.03,-0.07,0.21,0.03,-0.09,0.05,-0.08,-0.04,892.63,0.04,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.0,0.00,0.00,0.00
11,0.10,-0.04,0.00,0.02,0.06,-0.02,0.06,-0.02,0.04,533.41,...,0.00,0.00,0.00,-0.00,0.00,0.00,-0.0,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00001,0.00003,0.00000,0.00001,0.00003,0.00008,-0.00001,0.00003,...,0.00001,0.00001,-0.00001,0.00004,0.00003,-0.00000,0.00001,-0.00000,0.00001,-0.00000
3,-0.00001,1.00000,0.00000,-0.00002,0.00002,-0.00005,0.00001,0.00002,-0.00002,-0.00001,...,-0.00001,0.00002,0.00000,0.00002,0.00001,-0.00001,0.00001,-0.00001,-0.00001,-0.00000
4,0.00001,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00006,0.00000,...,-0.00003,0.00000,0.00002,-0.00002,0.00000,0.00001,0.00000,-0.00001,0.00001,0.00000
5,0.00003,-0.00002,0.00004,1.00000,-0.00004,0.00005,-0.00000,-0.00002,0.00001,0.00001,...,-0.00001,-0.00002,0.00001,-0.00000,0.00002,0.00000,0.00000,-0.00001,0.00001,0.00001
6,0.00000,0.00002,0.00002,-0.00004,1.00000,-0.00000,-0.00000,-0.00000,-0.00004,0.00003,...,0.00001,-0.00000,0.00001,0.00000,0.00001,0.00002,-0.00001,0.00001,-0.00001,-0.00000
7,0.00001,-0.00005,-0.00000,0.00005,-0.00000,1.00000,-0.00000,0.00000,0.00002,-0.00001,...,-0.00003,0.00001,-0.00002,0.00005,-0.00001,0.00002,0.00001,-0.00002,0.00000,-0.00001
8,0.00003,0.00001,0.00003,-0.00000,-0.00000,-0.00000,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00002,-0.00001,0.00000,-0.00000,0.00000,-0.00001,-0.00003,-0.00001,-0.00000
9,0.00008,0.00002,-0.00004,-0.00002,-0.00000,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,-0.00000,-0.00001,0.00003,-0.00001,0.00001,0.00000,0.00001,-0.00001,-0.00002
10,-0.00001,-0.00002,0.00006,0.00001,-0.00004,0.00002,-0.00005,-0.00003,1.00000,0.00005,...,0.00001,-0.00000,0.00000,-0.00001,0.00001,0.00001,-0.00001,0.00001,0.00001,0.00001
11,0.00003,-0.00001,0.00000,0.00001,0.00003,-0.00001,0.00005,-0.00002,0.00005,1.00000,...,0.00001,0.00001,0.00004,-0.00001,0.00003,0.00003,-0.00002,-0.00001,-0.00002,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

2.00326635372773

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.00288069e+07 1.84384448e+07 9.90382297e+06 4.83354648e+06
 2.05565100e+06 7.50215636e+05 2.87831180e+05 9.17043191e+04
 3.38261098e+04 1.20249907e+04 4.46799988e+03 1.68101577e+03
 6.29548707e+02 2.12570569e+02 6.91937958e+01 1.94155656e+01
 5.13582858e+00 9.82335176e-01 1.90897163e-01 2.86431607e-02
 6.13754214e-03 1.15432375e-03 1.59790869e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997817,-0.055644,-0.028595,-0.016957,-0.010233,-0.005969,-0.003652,-0.002039,-0.001252,-0.000744,...,-0.000100,-0.000058,-0.000031,-0.000015,-0.000007,-0.000003,-0.000001,-6.229022e-07,-2.758025e-07,-1.182865e-07
1,0.052505,0.995001,-0.075865,-0.032021,-0.017117,-0.009585,-0.005768,-0.003216,-0.001961,-0.001170,...,-0.000158,-0.000090,-0.000048,-0.000025,-0.000011,-0.000005,-0.000002,-1.022718e-06,-5.135455e-07,-1.793303e-07
2,0.030809,0.071150,0.993865,-0.071069,-0.028911,-0.014863,-0.008736,-0.004852,-0.002914,-0.001747,...,-0.000238,-0.000135,-0.000071,-0.000038,-0.000017,-0.000007,-0.000003,-1.469282e-06,-6.883116e-07,-2.626316e-07
3,0.019840,0.034526,0.065653,0.994608,-0.062893,-0.024977,-0.013861,-0.007498,-0.004496,-0.002676,...,-0.000362,-0.000208,-0.000110,-0.000058,-0.000025,-0.000011,-0.000004,-2.253399e-06,-1.014869e-06,-3.834927e-07
4,0.012712,0.019890,0.030165,0.058147,0.995620,-0.055164,-0.024689,-0.012470,-0.007373,-0.004339,...,-0.000582,-0.000336,-0.000178,-0.000093,-0.000041,-0.000018,-0.000007,-3.471749e-06,-1.829843e-06,-6.591373e-07
5,0.007736,0.011661,0.016408,0.025465,0.050902,0.995864,-0.061114,-0.023512,-0.013067,-0.007578,...,-0.001010,-0.000576,-0.000310,-0.000158,-0.000071,-0.000031,-0.000013,-6.300980e-06,-3.025702e-06,-1.146807e-06
6,0.005071,0.007535,0.010381,0.015312,0.025505,0.057374,0.995927,-0.053573,-0.024697,-0.013456,...,-0.001741,-0.000996,-0.000534,-0.000277,-0.000123,-0.000055,-0.000022,-1.098376e-05,-5.342059e-06,-1.941811e-06
7,0.002948,0.004372,0.005998,0.008643,0.013556,0.024266,0.049372,0.995596,-0.065687,-0.027637,...,-0.003211,-0.001842,-0.000984,-0.000508,-0.000227,-0.000101,-0.000040,-1.937690e-05,-9.802789e-06,-3.695266e-06
8,0.001936,0.002855,0.003874,0.005574,0.008632,0.014634,0.025451,0.061216,0.994645,-0.067922,...,-0.005715,-0.003268,-0.001741,-0.000906,-0.000400,-0.000180,-0.000072,-3.441312e-05,-1.705936e-05,-6.290353e-06
9,0.001225,0.001811,0.002463,0.003523,0.005408,0.009046,0.014909,0.028973,0.062331,0.993694,...,-0.010337,-0.005858,-0.003105,-0.001618,-0.000712,-0.000319,-0.000128,-6.192639e-05,-3.107939e-05,-1.131077e-05


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00218327461031953,
 0.004998560259032558,
 0.006134906536202855,
 0.00539223932855204,
 0.0043798990736800825,
 0.0041355032955640025,
 0.004073015077377784,
 0.004404461993927189,
 0.0053546834656659215,
 0.006306212726377991,
 0.007702913046889415,
 0.00910719731348153,
 0.00986866382446161,
 0.010425240184687379,
 0.010878459345381009,
 0.01138206744938608,
 0.011271182196771368,
 0.012209233954247556,
 0.015238929696850367,
 0.029895434706871504,
 0.06589164934557634,
 0.11335891016472899,
 0.07932344871009367]